In [1]:
# let's create a simple blockchain

In [2]:
import time
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64

In [3]:
class block:
    def __init__(self, prev_hash, data, nonce=None):
        # content of the block (e.g., transactions list)
        self.data = data
        
        # block header
        self.prev_hash = prev_hash
        self.timestamp = str(int(time.time()))
        if nonce:
            self.nonce = nonce
        else:
            self.nonce = base64.b16encode(os.urandom(16))
        
        # hash of the block header + data
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(self.prev_hash)
        digest.update(str.encode(self.timestamp))
        digest.update(self.nonce)
        digest.update(self.data)
        self.hash = base64.b16encode(digest.finalize())
    
    def find_hash(self):
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(self.prev_hash)
        digest.update(str.encode(self.timestamp))
        digest.update(self.nonce)
        digest.update(self.data)
        return base64.b16encode(digest.finalize())
    
    
    def find_nonce(self, difficulty):
        prefix = b'0' * difficulty
        while True:
            self.nonce = os.urandom(16)
            msg_digest = self.find_hash()
            if msg_digest.startswith(prefix):
                self.hash = msg_digest
                break
        
        
    def __repr__(self):
        return 'PreviousHash: {}\nTimestamp: {}\nNonce: {}\nHash {}'.format(
            self.prev_hash, self.timestamp, self.nonce, self.hash)
        

In [4]:
test_block = block(b'1', b'dummy data')

In [5]:
test_block

PreviousHash: b'1'
Timestamp: 1528383756
Nonce: b'BA3943E03BB8A911642E2C959D843008'
Hash b'E7817F842FD1CD8F777CDDA32C799E3B64759F36F1427DC8B3A6BD6E42153609'

In [6]:
genesis_block = block(
b'0000000000000000000000000000000000000000000000000000000000000000',
b'PyCon 2018 Genesis Block',
b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF')

In [7]:
genesis_block

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'

In [8]:
class blockchain:
    def __init__(self):
        self.blocks = []
        
    def add_block(self, block):
        self.blocks.append(block)
        
    def blockchain_valid(self):
        if self.blocks[0] != genesis_block:
            return False
        prev_block = self.blocks[0]
        
        for block in self.blocks[1:]:
            # Check if the prv_hash of the block points to the prv block
            if prev_block.hash != block.prev_hash:
                return False
            
            # Check Hash(current+prv) = current hash
            digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
            digest.update(block.prev_hash)
            digest.update(str.encode(block.timestamp))
            digest.update(block.nonce)
            digest.update(block.data)
            hash_digest = base64.b16encode(digest.finalize())
            
            if hash_digest != block.hash:
                return False
            
            prev_block = block

        return True
    
    def __repr__(self):
        _blocks = []
        for _block in self.blocks:
            _blocks.append(str(_block))
        return '\n\n'.join(_blocks)

In [9]:
test_blockchain = blockchain()

In [10]:
test_blockchain

In [11]:
test_blockchain.add_block(genesis_block)

In [12]:
test_blockchain.blocks

[PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
 Timestamp: 1528383756
 Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
 Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538']

In [13]:
test_blockchain

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'

In [14]:
print(test_blockchain.blockchain_valid())

True


In [15]:
# bad genesis block
test_blockchain = blockchain()
test_blockchain.add_block(block(b'1', b'2'))
print(test_blockchain.blockchain_valid())

False


In [16]:
# corect blocks
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)
print(test_blockchain.blockchain_valid())

print(genesis_block)
test_block = block(genesis_block.hash, b'dummy data')
print(test_block)
test_blockchain.add_block(test_block)
test_blockchain.blockchain_valid()

True
PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'
PreviousHash: b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'
Timestamp: 1528383756
Nonce: b'CFE2682114589D99F0F89EC32DE8EDE8'
Hash b'D4A07A28FBAF211C46440408E46252139A01BBF60557D1ABEE433180A58D2B0C'


True

In [17]:
test_block2 = block(test_block.hash, b'dummy data2')
print(test_block2)
test_blockchain.add_block(test_block2)
test_blockchain.blockchain_valid()

PreviousHash: b'D4A07A28FBAF211C46440408E46252139A01BBF60557D1ABEE433180A58D2B0C'
Timestamp: 1528383756
Nonce: b'19EFF1DE5590BB20DCA572FD28760A30'
Hash b'0546D307EEAC0061AA82C3A0742BC6145DA4DBB88EA09DB720462A50A75FFE66'


True

In [18]:
test_blockchain

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'

PreviousHash: b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'
Timestamp: 1528383756
Nonce: b'CFE2682114589D99F0F89EC32DE8EDE8'
Hash b'D4A07A28FBAF211C46440408E46252139A01BBF60557D1ABEE433180A58D2B0C'

PreviousHash: b'D4A07A28FBAF211C46440408E46252139A01BBF60557D1ABEE433180A58D2B0C'
Timestamp: 1528383756
Nonce: b'19EFF1DE5590BB20DCA572FD28760A30'
Hash b'0546D307EEAC0061AA82C3A0742BC6145DA4DBB88EA09DB720462A50A75FFE66'

In [19]:
# proof of work

In [20]:
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)

In [21]:
test_blockchain

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'

In [22]:
test_block = block(genesis_block.hash, b'dd')
test_block

PreviousHash: b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'
Timestamp: 1528383756
Nonce: b'6B972DAFD2917C0473385909EAC1BADF'
Hash b'D4D9F6F8B5482F3AEDAC3174B08BCF46799390C892FAA02AA81805FA8F3F302D'

In [23]:
test_block.find_nonce(3)
test_block

PreviousHash: b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'
Timestamp: 1528383756
Nonce: b'\xb0\xc4\x84\xbc\x00\xf6\x1e\xca=\x8f\x8cq\x07I\x1f\x9c'
Hash b'000BF26C2806AC5B79CD87C86C6515E75A72EE6DB38584F19CC0368B9CA4D40B'

In [24]:
# tale of two blockchains

In [25]:
test_blockchain1 = blockchain()
test_blockchain2 = blockchain()

test_blockchain1.add_block(genesis_block)
test_blockchain2.add_block(genesis_block)

print(test_blockchain1)
print()
print(test_blockchain1)

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'

PreviousHash: b'0000000000000000000000000000000000000000000000000000000000000000'
Timestamp: 1528383756
Nonce: b'FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF'
Hash b'5CAC26C6B1146437CCA06E6D4C3AEF0F0D9EFA3B8877F7AC3E00DFA4D15CA538'


In [26]:
def mine_blocks(test_blockchain):
    s_time = int(time.time())
    duration = 5
    
    _prev_block = genesis_block
    
    while True:
        _block = block(_prev_block.hash, os.urandom(4))
        _block.find_nonce(3)
        test_blockchain.add_block(_block)
        print('.', end='')
        if int(time.time()) > duration + s_time:
            break
    

In [27]:
mine_blocks(test_blockchain1)

..........................................................................................

In [28]:
mine_blocks(test_blockchain2)

.............................................................................

In [29]:
len(test_blockchain1.blocks)

91

In [30]:
len(test_blockchain2.blocks)

78

In [31]:
# asymmetric encryption system and signatures

In [32]:
# tx data from, to, amount
tx_data = '{0}======={1}======={2}'

In [33]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.serialization import PublicFormat, Encoding
from cryptography.hazmat.primitives import serialization

In [34]:
def hash_pub_key(private_key):
    digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
    digest.update(private_key.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo))
    return base64.b16encode(digest.finalize())

def sign_tx(tx, private_key):
    return private_key.sign(tx, ec.ECDSA(hashes.SHA256()))

def serialize_pubkey(publickey):
    serialized_public = publickey.public_bytes(
    encoding=Encoding.PEM,
    format=PublicFormat.SubjectPublicKeyInfo)
    
    return serialized_public

def parse_serialized_pubkey(serialized_pubkey):
    loaded_public_key = serialization.load_pem_public_key(serialized_pubkey, backend=default_backend())
    return loaded_public_key

In [35]:
private_keys = {}
private_keys['Alice'] = ec.generate_private_key(ec.SECP256K1(), default_backend())
private_keys['Bob'] = ec.generate_private_key(ec.SECP256K1(), default_backend())
private_keys['Carol'] = ec.generate_private_key(ec.SECP256K1(), default_backend())

In [36]:
private_keys

{'Alice': <cryptography.hazmat.backends.openssl.ec._EllipticCurvePrivateKey at 0x10bbcb748>,
 'Bob': <cryptography.hazmat.backends.openssl.ec._EllipticCurvePrivateKey at 0x10bbbba58>,
 'Carol': <cryptography.hazmat.backends.openssl.ec._EllipticCurvePrivateKey at 0x10bbbbb70>}

In [37]:
# Alice -> Bob, 10$
tx_data_b = str.encode(tx_data.format(hash_pub_key(private_keys['Alice']), 
                                      hash_pub_key(private_keys['Bob']), 
                                      '10'))
serialize_pubkey = serialize_pubkey(private_keys['Alice'].public_key())
signature = sign_tx(tx_data_b, private_keys['Alice'])

In [38]:
tx_b = tx_data_b+b'======='+serialize_pubkey+b'======='+signature

In [39]:
tx_b

b"b'A7EE2B91A4B77D084845EB17B014E00EAA2A488B65BAD21B6594E69A94BE8C38'=======b'32F3D166E6BBB2BA7E74DD5D6358A5FDDAF04A827582CFFD56B9E2E4F861C4E6'=======10=======-----BEGIN PUBLIC KEY-----\nMFYwEAYHKoZIzj0CAQYFK4EEAAoDQgAEhnTYy8vqyEwrN7YsekHu5EfdwglF3PwN\n+HL+LUCJ2jz9xxOXm6rcejbLD2vsm0ec0gStEp+VDA3lf+sMssOcvQ==\n-----END PUBLIC KEY-----\n=======0E\x02!\x00\xb07-\x8du6\x15\xd0\xe9\xd3\x81\x1b\x1f\xdba\xea\x17\xf4\xd7\x0c\xad&D}q\xc0\xe7\xdb\x7fk(\xad\x02 \x1e\xf8Z\xb7e\x87J3\xca\x11)\x9e\xf6\xbeh-\x14\x04\x98\x97\x8b\xb4z2\x12\x9e\xee\xd1%\xf1\xba\xd5"

In [40]:
# Bob -> Carol, 15$
tx_data_b = str.encode(tx_data.format(hash_pub_key(private_keys["Bob"]),
                          hash_pub_key(private_keys["Carol"]),
                          "15"))
serialized_pubkey = serialize_pubkey(private_keys["Bob"].public_key())
siganture = sign_tx(tx_data_b, private_keys["Bob"])

TypeError: 'bytes' object is not callable

In [ ]:
# transaction data
tx_b2 = tx_data_b+b'======='+serialized_pubkey+b'======='+siganture

In [ ]:
# # bob -> carol, 15$
# tx_data_b = str.encode(tx_data.format(hash_pub_key(private_keys['Bob']), hash_pub_key(private_keys['Alice']), '15'))
# serialize_pubkey = serialize_pubkey(private_keys['Bob'].public_key())

In [ ]:
def verify_tx_signatures(tx):
    pubkey = parse_serialized_pubkey(tx.split(b'=======')[3])
    signature = tx.split(b'=======')[4]
    data = b'======='.join(tx.split(b'=======')[:3])
    pubkey.verify(signature, data, ec.ECDSA(hashes.SHA256()))
    print('OK')

In [ ]:
test_blockchain = blockchain()
test_blockchain.add_block(genesis_block)

In [ ]:
test_block = block(genesis_block.hash, tx_b)
test_block.find_nonce(3)
test_blockchain.add_block(test_block)

test_block = block(test_block.hash, tx_b2)
test_block.find_nonce(3)
test_blockchain.add_block(test_block)

tset_blockchain.blockchain_valid()